In [2]:
import utils
import os
import requests
from urllib.parse import urljoin
import time
import utils


In [3]:
os.environ['EXTENDED_API_HOST']='https://domino-extensions-api-svc.domino-field.svc.cluster.local'
#Create user SA's
EXTENDED_API_URI = os.environ.get('EXTENDED_API_HOST')

manage_sa_url = urljoin(EXTENDED_API_URI, 'domino-sa-management-api/usersa')
get_all_sa_url = urljoin(EXTENDED_API_URI, 'domino-sa-management-api/allusersa')

In [4]:
def get_all_users():
    headers = utils.generate_api_key_based_request_headers()
    url: str = urljoin(os.environ.get('DOMINO_API_HOST'), '/api/users/v1/users')
    users = requests.get(url, headers=headers).json()
    return users
def get_user(user_name:str):
    users = get_all_users()
    for u in users['users']:        
        name = u['userName']
        if name==user_name:
            return u
    return None

## Get All User Based SA's in the domino-compute namespace

First determine if you have any user based SA's for a domino user

In [8]:
import pandas as pd
import json
headers = utils.generate_api_key_based_request_headers()
response = requests.get(get_all_sa_url, headers=headers,  verify=False)
lst = response.json()['all_sa']
print(f'Total number of user based k8s sa in the domino-compute namespace = {len(lst)}')
df = pd.DataFrame(lst)
df

Total number of user based k8s sa in the domino-compute namespace = 5


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,domino_user_id,domino_user_name,sa_name
0,65a98234e7e76d34e50712be,integration-test,integration-test
1,65be5199e174df651910cc5a,sameer_wadkar,sameer-wadkar
2,65bfa482e174df651910ccd8,sameer+wadkar@dominodatalab.com,sameer-wadkar-dominodatalab.com
3,65be51c7e174df651910cc70,sameer.wadkar@dominodatalab.com,sameer.wadkar-dominodatalab.com
4,65be51aee174df651910cc65,sameerwadkar,sameerwadkar


# Generate user SA for all users

Generate SA for all domino users in the `domino-compute` namespace

In [13]:
headers = utils.generate_api_key_based_request_headers()
users = get_all_users()
for u in users['users']:
    id = u['id']
    name = u['userName']
    body = {'user_name': name, 'user_id': id, 'domino_sa': None}
    print(f'Generating K8s SA for user {body} ')
    response = requests.post(manage_sa_url, headers=headers, json=body, verify=False)
    if response.status_code==200:
        print('Success')
        print(response.status_code)
        print(response.text)
    else:
        print('Failure')
        print(response.status_code)
        print(response.text)        
    print('\n')

Generating K8s SA for user {'user_name': 'sameer+wadkar@dominodatalab.com', 'user_id': '65bfa482e174df651910ccd8', 'domino_sa': None} 
Success
200
SA=sameer-wadkar-dominodatalab.com generated for domino_user=sameer+wadkar@dominodatalab.com


Generating K8s SA for user {'user_name': 'sameer.wadkar@dominodatalab.com', 'user_id': '65be51c7e174df651910cc70', 'domino_sa': None} 
Success
200
SA=sameer.wadkar-dominodatalab.com generated for domino_user=sameer.wadkar@dominodatalab.com


Generating K8s SA for user {'user_name': 'sameerwadkar', 'user_id': '65be51aee174df651910cc65', 'domino_sa': None} 


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

Success
200
SA=sameerwadkar generated for domino_user=sameerwadkar


Generating K8s SA for user {'user_name': 'sameer_wadkar', 'user_id': '65be5199e174df651910cc5a', 'domino_sa': None} 
Success
200
SA=sameer-wadkar generated for domino_user=sameer_wadkar


Generating K8s SA for user {'user_name': 'sameer-wadkar', 'user_id': '65be5180e174df651910cc4f', 'domino_sa': None} 
Failure
409
SA sameer-wadkar already exists for sameer-wadkar. First delete and then recreate


Generating K8s SA for user {'user_name': 'integration-test', 'user_id': '65a98234e7e76d34e50712be', 'domino_sa': None} 


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

Success
200
SA=integration-test generated for domino_user=integration-test




## Delete All User Based SA's

Remove all SA for all domino users in the `domino-compute` namespace

In [12]:
#Delete user SA's
headers = utils.generate_api_key_based_request_headers()
users = get_all_users()
for u in users['users']:
    name = u['userName']
    body = {'user_name': name}
    print(f'Delete url {manage_sa_url}')
    print(f'Deleting K8s SA for user {name}')

    response = requests.delete(manage_sa_url, headers=headers, json=body, verify=False)


    print(response.status_code)
    print(response.text)

Delete url https://domino-extensions-api-svc.domino-field.svc.cluster.local/domino-sa-management-api/usersa
Deleting K8s SA for user sameer+wadkar@dominodatalab.com
200
SA sameer-wadkar-dominodatalab.com  for  Domino User sameer+wadkar@dominodatalab.com deleted
Delete url https://domino-extensions-api-svc.domino-field.svc.cluster.local/domino-sa-management-api/usersa
Deleting K8s SA for user sameer.wadkar@dominodatalab.com
200
SA sameer.wadkar-dominodatalab.com  for  Domino User sameer.wadkar@dominodatalab.com deleted
Delete url https://domino-extensions-api-svc.domino-field.svc.cluster.local/domino-sa-management-api/usersa
Deleting K8s SA for user sameerwadkar


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

200
SA sameerwadkar  for  Domino User sameerwadkar deleted
Delete url https://domino-extensions-api-svc.domino-field.svc.cluster.local/domino-sa-management-api/usersa
Deleting K8s SA for user sameer_wadkar
200
SA sameer-wadkar  for  Domino User sameer_wadkar deleted
Delete url https://domino-extensions-api-svc.domino-field.svc.cluster.local/domino-sa-management-api/usersa
Deleting K8s SA for user sameer-wadkar
404
SA does not exist for Domino User sameer-wadkar
Delete url https://domino-extensions-api-svc.domino-field.svc.cluster.local/domino-sa-management-api/usersa
Deleting K8s SA for user integration-test
200
SA integration-test  for  Domino User integration-test deleted


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

# Example of overriding the Default Service Account by admin for a user

Domino allows user names which are valid email addresses. Therefore the following are perfectly valid user-names
- sameer-wadkar@dominodatalab.com
- sameer+wadkar@dominodatalab.com

Both translate into the same service-account - `sameer-wadkar-dominodatalab.com` which is problematic. The service will not allow you to create the next one citing that there is a duplicate SA. The Admin has the ability to override in  such cases

In [ ]:
headers = utils.generate_api_key_based_request_headers()
user_name = 'sameer+wadkar@dominodatalab.com'
u = get_user(user_name)
if u is not None:
    body = {'user_name': user_name, 'user_id': u['id'], 'domino_sa':'sameer-wadkar-dominodatalab.com2'}        
    response = requests.post(manage_sa_url, headers=headers, json=body, verify=False)
    print(response.status_code)
    print(response.text)

In [ ]:
# Get SA for a single domino user

In [ ]:
get_one_sa_url = urljoin(EXTENDED_API_URI, 'domino-sa-management-api/usersa')
body = {'user_name': 'sameerwadkar'}
response = requests.get(manage_sa_url, headers=headers, json=body, verify=False)
print(response.status_code)
print(response.text)